In [1]:
!pip install tqdm

In [2]:
# Importar las bibliotecas necesarias
import requests
from bs4 import BeautifulSoup

from google.colab import drive
import time
import random
from tqdm.notebook import tqdm

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from tqdm import tqdm
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Función para extraer información de una tarjeta
def extract_card_info(card):
    title_element = card.select_one('.article-title')
    title = title_element.text.strip()
    url = "https://dzone.com" + title_element['href']
    detail = card.select_one('.article-desc').text.strip()
    date = card.select_one('.article-date').contents[0].strip()
    author = card.select_one('.article-author').text.strip()
    views = card.select_one('.article-stats').text.split('·')[1].strip()
    return {
        'Title': title,
        'URL': url,
        'Detail': detail,
        'Date': date,
        'Author': author,
        'Views': views
    }

def scrape_dzone():
    base_url = "https://dzone.com/software-design-and-architecture/"
    csv_path = '/content/drive/MyDrive/dzone_software_design_architecture.csv'

    # Verificar si el CSV existe y obtener la última página procesada
    try:
        df_existing = pd.read_csv(csv_path)
        last_processed_page = len(df_existing) // 25 + 1
    except FileNotFoundError:
        last_processed_page = 2

    # Calcular el número total de páginas
    total_pages = 648

    # Crear la barra de progreso
    with tqdm(total=total_pages, initial=last_processed_page-2, desc="Scraping Progress") as pbar:
        for page in range(last_processed_page, total_pages + 1):
            url = f"{base_url}{page}"

            try:
                response = requests.get(url)
                response.raise_for_status()
                soup = BeautifulSoup(response.content, 'html.parser')

                cards = soup.select('div.media')

                page_data = []
                for card in cards:
                    card_info = extract_card_info(card)
                    page_data.append(card_info)

                # Convertir los datos de la página a DataFrame
                df_page = pd.DataFrame(page_data)

                # Guardar o anexar al CSV
                if page == 2 and last_processed_page == 2:
                    df_page.to_csv(csv_path, index=False, mode='w')
                else:
                    df_page.to_csv(csv_path, index=False, mode='a', header=False)

                # Actualizar la barra de progreso con información adicional
                pbar.set_postfix_str(f"Page: {page}, URL: {url}")
                pbar.update(1)

            except Exception as e:
                print(f"Error processing page {url}: {e}")

            # Pausa aleatoria para evitar sobrecarga del servidor
            time.sleep(random.uniform(1, 3))

    print(f"Scraping completo. CSV guardado en: {csv_path}")

# Ejecutar el scraping
scrape_dzone()

# Cargar el CSV final y mostrar información
df = pd.read_csv('/content/drive/MyDrive/dzone_software_design_architecture.csv')

# Mostrar las primeras filas del DataFrame
print(df.head())

# Mostrar información sobre el DataFrame
print(df.info())

Mounted at /content/drive


Scraping Progress: 100%|█████████▉| 647/648 [32:30<00:03,  3.01s/it, Page: 648, URL: https://dzone.com/software-design-and-architecture/648]


Scraping completo. CSV guardado en: /content/drive/MyDrive/dzone_software_design_architecture.csv
                                               Title  \
0  Serverless Computing and GraphQL: Modern App D...   
1  CI/CD Pipelines in the Cloud: How Cloud Hostin...   
2  What We Learned About Secrets Security at AppS...   
3  Mutable vs. Immutable: Infrastructure Models i...   
4               An Overview of TCPCopy for Beginners   

                                                 URL  \
0  https://dzone.com/articles/serverless-computin...   
1  https://dzone.com/articles/cicd-pipelines-in-t...   
2  https://dzone.com/articles/secrets-security-at...   
3  https://dzone.com/articles/infrastructure-mode...   
4  https://dzone.com/articles/an-overview-of-tcpc...   

                                              Detail              Date  \
0  In this step-by-step guide, learn how to defin...  October 14, 2024   
1  Cloud-hosted CI/CD pipelines enhance software ...  October 14, 2024   
2  Lea

In [ ]:
# Montar Google Drive
from google.colab import drive
import pandas as pd

# Montar Google Drive
drive.mount('/content/drive')

# Ruta del archivo CSV original en Google Drive
csv_path = '/content/drive/MyDrive/dzone_architecture_design_news.csv'

# Cargar el archivo CSV
df = pd.read_csv(csv_path)

# Eliminar filas donde Tag, Author y Date sean NaN
df_cleaned = df.dropna(subset=['Tag', 'Author', 'Date'], how='all')

# Eliminar filas completamente vacías (que tienen solo valores NaN o están en blanco)
df_cleaned = df_cleaned.dropna(how='all')

# Guardar el DataFrame limpio en un nuevo archivo CSV
cleaned_csv_path = '/content/drive/MyDrive/dzone_architecture_design_news_cleaned.csv'
df_cleaned.to_csv(cleaned_csv_path, index=False)

# Mostrar las primeras filas del DataFrame limpio
print(df_cleaned.head(100))

print(f"CSV limpio guardado en: {cleaned_csv_path}")

In [11]:


# Montar Google Drive
from google.colab import drive
import pandas as pd
import re
from datetime import datetime

# Montar Google Drive
drive.mount('/content/drive')

# Ruta del archivo CSV original en Google Drive
csv_path = '/content/drive/MyDrive/dzone_software_design_architecture.csv'


# Cargar el archivo CSV
df = pd.read_csv(csv_path)

def clean_date(date_str):
    original_date = date_str
    # Eliminar 'Updated' y cualquier fecha adicional
    date_str = re.split(r'Updated', date_str)[-1].strip()
    date_str = re.split(r',', date_str)[0].strip()

    # Buscar el año en la cadena original
    year_match = re.search(r'\b\d{4}\b', original_date)
    year = year_match.group() if year_match else ""

    # Convertir el formato de la fecha
    try:
        date_obj = datetime.strptime(date_str, '%B %d')
        return f"{date_obj.strftime('%b %d').lower()} {year}", original_date
    except ValueError:
        try:
            date_obj = datetime.strptime(date_str, '%b %d')
            return f"{date_obj.strftime('%b %d').lower()} {year}", original_date
        except ValueError:
            # Si no se puede parsear, intentar extraer mes y día
            month_day_match = re.search(r'([A-Za-z]+)\s+(\d{1,2})', date_str)
            if month_day_match and year:
                month, day = month_day_match.groups()
                return f"{month[:3].lower()} {day.zfill(2)} {year}", original_date
            else:
                return date_str, original_date  # Devolver la cadena original si no se puede parsear

# Aplicar la función de limpieza a la columna 'Date'
df['CleanedDate'], df['OriginalDate'] = zip(*df['Date'].apply(clean_date))

# Guardar el DataFrame con la columna 'Date' limpia en un nuevo archivo CSV
cleaned_csv_path = '/content/drive/MyDrive/dzone_architecture_design_news_date_cleaned.csv'
df.to_csv(cleaned_csv_path, index=False)

# Mostrar las primeras filas del DataFrame con la columna 'Date' original y limpia
print(df[['OriginalDate', 'CleanedDate']].head(20))

print(f"CSV con la columna 'Date' limpia guardado en: {cleaned_csv_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
               OriginalDate  CleanedDate
0          October 14, 2024  oct 14 2024
1          October 14, 2024  oct 14 2024
2          October 14, 2024  oct 14 2024
3          October 14, 2024  oct 14 2024
4          October 11, 2024  oct 11 2024
5          October 11, 2024  oct 11 2024
6          October 10, 2024  oct 10 2024
7          October 10, 2024  oct 10 2024
8           October 9, 2024  oct 09 2024
9           October 9, 2024  oct 09 2024
10          October 9, 2024  oct 09 2024
11          October 9, 2024  oct 09 2024
12  Updated October 9, 2024  oct 09 2024
13          October 9, 2024  oct 09 2024
14          October 9, 2024  oct 09 2024
15          October 8, 2024  oct 08 2024
16          October 8, 2024  oct 08 2024
17          October 8, 2024  oct 08 2024
18          October 8, 2024  oct 08 2024
19          October 8, 2024  oct 08 2024
CSV con la

In [13]:
# Montar Google Drive
from google.colab import drive
import pandas as pd
import re
from datetime import datetime

# Montar Google Drive
drive.mount('/content/drive')

# Ruta del archivo CSV original en Google Drive
csv_path = '/content/drive/MyDrive/dzone_software_design_architecture.csv'


# Cargar el archivo CSV
df = pd.read_csv(csv_path)

def clean_date(date_str):
    original_date = date_str
    # Eliminar 'Updated' y cualquier fecha adicional
    date_str = re.split(r'Updated', date_str)[-1].strip()
    date_str = re.split(r',', date_str)[0].strip()

    # Buscar el año en la cadena original
    year_match = re.search(r'\b\d{4}\b', original_date)
    year = year_match.group() if year_match else ""

    # Convertir el formato de la fecha
    try:
        date_obj = datetime.strptime(date_str, '%B %d')
        return f"{date_obj.strftime('%b %d').lower()} {year}", original_date
    except ValueError:
        try:
            date_obj = datetime.strptime(date_str, '%b %d')
            return f"{date_obj.strftime('%b %d').lower()} {year}", original_date
        except ValueError:
            # Si no se puede parsear, intentar extraer mes y día
            month_day_match = re.search(r'([A-Za-z]+)\s+(\d{1,2})', date_str)
            if month_day_match and year:
                month, day = month_day_match.groups()
                return f"{month[:3].lower()} {day.zfill(2)} {year}", original_date
            else:
                return date_str, original_date  # Devolver la cadena original si no se puede parsear

# Aplicar la función de limpieza a la columna 'Date'
df['CleanedDate'], df['OriginalDate'] = zip(*df['Date'].apply(clean_date))

# Guardar la fecha limpia en la columna 'Date' original
df['Date'] = df['CleanedDate']

# Eliminar las columnas temporales
df = df.drop(columns=['CleanedDate', 'OriginalDate'])

# Guardar el DataFrame con la columna 'Date' limpia en un nuevo archivo CSV
cleaned_csv_path = '/content/drive/MyDrive/dzone_architecture_design_news_date_cleaned.csv'
df.to_csv(cleaned_csv_path, index=False)

# Mostrar las primeras filas del DataFrame con la columna 'Date' limpia
print(df[['Date']].head(20))

print(f"CSV con la columna 'Date' limpia guardado en: {cleaned_csv_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
           Date
0   oct 14 2024
1   oct 14 2024
2   oct 14 2024
3   oct 14 2024
4   oct 11 2024
5   oct 11 2024
6   oct 10 2024
7   oct 10 2024
8   oct 09 2024
9   oct 09 2024
10  oct 09 2024
11  oct 09 2024
12  oct 09 2024
13  oct 09 2024
14  oct 09 2024
15  oct 08 2024
16  oct 08 2024
17  oct 08 2024
18  oct 08 2024
19  oct 08 2024
CSV con la columna 'Date' limpia guardado en: /content/drive/MyDrive/dzone_architecture_design_news_date_cleaned.csv


In [ ]:
# Montar Google Drive
drive.mount('/content/drive')

# Cargar el CSV final y mostrar información
df = pd.read_csv('/content/drive/MyDrive/infoq_architecture_design_news.csv')

# Mostrar las primeras filas del DataFrame
print(df.head())

# Mostrar información sobre el DataFrame
print(df.info())

Mounted at /content/drive


NameError: name 'pd' is not defined